In [1]:
import pandas as pd
import sqlite3

In [2]:
# Define function to execute queries
def execute_query(database_path, query):
    """Execute a SQL query on a given database and return the results."""
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    conn.close()
    return results

# Query for Deliveroo
query_deli = """
SELECT 
    restaurants.name, 
    restaurants.rating, 
    AVG(menu_items.price) AS avg_price,  
    ROUND(AVG(menu_items.price) / restaurants.rating, 2) AS price_to_rating_ratio,
    'Deliveroo'
FROM 
    restaurants
INNER JOIN 
    menu_items 
ON 
    restaurants.id = menu_items.restaurant_id
WHERE 
    restaurants.rating > 0.1 
GROUP BY 
    restaurants.id
HAVING 
    AVG(menu_items.price) > 10  -- Filter by average price > 10 to filter out sandwich shops and strage values
ORDER BY 
    price_to_rating_ratio ASC;  
"""

# Query for Takeaway
query_take = """
SELECT 
    restaurants.name,
    restaurants.ratings,
    AVG(menuItems.price) AS avg_price,
    ROUND(AVG(menuItems.price) / restaurants.ratings, 2) AS price_to_rating_ratio,
    'Takeaway' 
FROM 
    restaurants
INNER JOIN 
    menuItems 
ON 
    restaurants.primarySlug = menuItems.primarySlug
WHERE 
    restaurants.ratings > 0.1 
GROUP BY 
    menuItems.primarySlug
HAVING 
    AVG(menuItems.price) > 10   -- Filter by average price > 10 to filter out sandwich shops and strage values
ORDER BY 
    price_to_rating_ratio ASC;  
"""

# Query for UberEats
query_uber = """
SELECT 
    restaurants.title,
    restaurants.rating__rating_value,
    AVG(menu_items.price) / 100 AS avg_price,   -- menu_items.price is in eurocent
    ROUND(AVG(menu_items.price) / 100 / restaurants.rating__rating_value, 2) AS price_to_rating_ratio,
    'Ubereats' 
FROM restaurants
INNER JOIN 
    menu_items 
ON 
    restaurants.id = menu_items.restaurant_id
WHERE 
    restaurants.rating__rating_value > 0.1 
GROUP BY 
    menu_items.restaurant_id
HAVING 
    AVG(menu_items.price) / 100 > 10     -- menu_items.price is in eurocent. Filter by average price > 10 to filter out sandwich shops and strage values
ORDER BY 
    price_to_rating_ratio ASC; 
"""

# Execute queries and get results
results_deli = execute_query("../Databases/deliveroo.db", query_deli)
results_take = execute_query("../Databases/takeaway.db", query_take)
results_uber = execute_query("../Databases/ubereats.db", query_uber)

# Combine results into one list and create a DataFrame
combined_results = results_deli + results_take + results_uber
columns = ['Restaurant Name', 'Rating', 'Avg Price', 'Price-to-Rating Ratio', 'Platform']
df = pd.DataFrame(combined_results, columns=columns)

# Sort by Price-to-Rating Ratio
df = df.sort_values(by='Price-to-Rating Ratio', ascending=True)


# Display the result
print(df)




               Restaurant Name  Rating  Avg Price  Price-to-Rating Ratio  \
459                       Bite     4.9  10.215789                   2.08   
461          Le Pain Quotidien     4.8  10.042808                   2.09   
462                   Mozaique     4.8  10.046269                   2.09   
460             Restaurant Gok     4.8  10.041667                   2.09   
463             Pizzeria Tiamo     4.8  10.096471                   2.10   
...                        ...     ...        ...                    ...   
2080  Japans Restaurant Amatsu     4.4  38.469697                   8.74   
456                Lobster Pot     4.1  35.958333                   8.77   
2081        Pizza Hut Delivery     1.9  17.344944                   9.13   
457            Neuhaus Hasselt     4.2  44.307692                  10.55   
458         Ottoman grillhouse     4.2  52.953704                  12.61   

       Platform  
459    Takeaway  
461    Takeaway  
462    Takeaway  
460    Takeaway